In [ ]:
import globus_sdk
import os
import spectral_deconvolution
import sys
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
#authenticate into globus

CLIENT_ID = 'YOUR_GLOBUS_CLIENT_ID'
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()
authorize_url = client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")

In [ ]:
auth_code = input("Please enter the code you get after login here: ").strip()

token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server["auth.globus.org"]
globus_transfer_data = token_response.by_resource_server["transfer.api.globus.org"]

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data["access_token"]
TRANSFER_TOKEN = globus_transfer_data["access_token"]

In [ ]:
#globus transfer endpoints
matrix_completion_public = 'bfd43535-052a-46bb-911a-b1cdfb1e92d7'
my_local_endpoint = 'YOUR_GLOBUS_ENDPOINT_UUID'
local_globus = 'FILE_PATH_TO_YOUR_LOCAL_GLOBUS'

authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
transfer_client = globus_sdk.TransferClient(authorizer=authorizer)


In [ ]:
#transfer the spectral deconvolution folder to local

source_path = '/spectral_deconvolution/matrix_completion'
dest_path = "/" + os.path.join(*local_globus.split("/"),*source_path.split("/"))

tdata = globus_sdk.TransferData(transfer_client,
                                matrix_completion_public,
                                my_local_endpoint,
                                label="spectral deconvolution",
                                sync_level="checksum")


tdata.add_item(source_path, dest_path, recursive=True)

# Start the transfer and print the resulting task ID
transfer_result = transfer_client.submit_transfer(tdata)
print("Transfer Task ID:", transfer_result['task_id'])


In [ ]:
#load the spectral deconvolution file data
spectral_deconvolution_data = pd.read_csv(os.path.join(dest_path,'spectral_deconvolution_data.csv'))

In [ ]:
#transfer mzml files for spectral deconvolution

tdata = globus_sdk.TransferData(transfer_client,
                                matrix_completion_public,
                                my_local_endpoint,
                                label="spectral deconvolution mzml",
                                sync_level="checksum")

globus_paths = []
for i,fname in enumerate(spectral_deconvolution_data['mzml_path']):
    
    dest_path = "/" + os.path.join(*local_globus.split("/"),*fname.lower().split("/"))
    tdata.add_item(fname.lower(), dest_path)

transfer_result = transfer_client.submit_transfer(tdata)
print("Transfer Task ID:", transfer_result['task_id'])

In [ ]:
#run spectral deconvolution from interactive or command line
output_path = "/" + os.path.join(*local_globus.split("/"),'spectral_deconvolution','matrix_completion')
calibration_path = "/" + os.path.join(*local_globus.split("/"),'spectral_deconvolution','matrix_completion','calibration_data')
for i,fname in enumerate(spectral_deconvolution_data['mzml_path']):
    mzml_path = "/" + os.path.join(*local_globus.split("/"),*fname.lower().split("/"))
    
    if os.path.exists(mzml_path):
        sys.argv = [
            'spectral_deconvolution.py', #script
            mzml_path,
            output_path, #output folder path
            1.666, #max concentration in reaction plate, in mM
            calibration_path
        ]

        spectral_deconvolution.main()
